In [1]:
import pandas as pd
import numpy as np
from scipy import linalg
import gensim
from utility_functions import *
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

pd.set_option('display.width', 1000)

In [2]:
df = pd.read_csv(f"../data/1 - Original data/PreprocessedData.csv")
df.head()

,PersonID,Speaker,QuoteText,NonStopwordsQuoteText,QuoteWordCount,QuoteLen,StartDate,$Type
0,48.0,מזכירת הכנסת ירדנה מלר-הורוביץ:,לקריאה שנייה ולקריאה שלישית: הצעת חוק ההתיישנו...,שנייה ולקריאה שלישית ההתיישנות 5 התשעה–2015 שה...,16,106,2015-07-28 16:00:00,Plenum
1,48.0,מזכירת הכנסת ירדנה מלר-הורוביץ:,"לקריאה ראשונה, מטעם הכנסת: הצעת חוק ההוצאה לפו...",מטעם ההוצאה לפועל 49 מניעת עיקול מיטלטלין המשמ...,25,147,2015-11-30 16:00:00,Plenum
2,48.0,מזכירת הכנסת ירדנה מלר-הורוביץ:,"ברשות יושב-ראש הכנסת, הנני מתכבדת להודיעכם, כי...",ברשות יושב-ראש הנני להודיעכם הונחו שולחן מסקנו...,29,176,2015-12-30 11:00:00,Plenum
3,488.0,"היו""ר יצחק וקנין:","כהרגלנו ביום רביעי, פותחים בשאילתות דחופות. שא...",כהרגלנו רביעי פותחים בשאילתות דחופות שאילתה דח...,36,190,2016-03-02 11:00:00,Plenum
4,30080.0,רחל עזריה (כולנו):,"בוקר טוב, צהריים טובים, תודה על ההזדמנות. בישר...",צהריים ההזדמנות בישראל מסורבות גט בתופעה רחבת ...,20,114,2016-03-02 11:00:00,Plenum


# Choose random samples
First, we choose random valid samples from the dataframe - valid samples are those that contain 15-150 words and belong to a relevant speaker (parliament member) and were not part of a meeting opening quotes.


In [ ]:
# Choosing random samples and saving them to csv
df = pd.read_csv(f"../data/1 - Original data/PreprocessedData.csv")
for sample_num in [15000, 20000, 25000]:
    for version in ['a', 'b', 'c']:
        samples_df = df.sample(n=sample_num)
        samples_df['NonStopwordsQuoteText'] = samples_df['NonStopwordsQuoteText'].astype("string")
        file_name = f"../data/2 - Trial data/{int(sample_num / 1000)}k_{version}_sample_quotes.csv"
        samples_df.to_csv(file_name)
        print(f"{int(sample_num / 1000)}k_{version} saved")

In [2]:
# 3 versions of 15k samples:
corpi_15_a = create_corpus("../data/2 - Trial data/15k_a_sample_quotes.csv")
corpi_15_b = create_corpus("../data/2 - Trial data/15k_b_sample_quotes.csv")
corpi_15_c = create_corpus("../data/2 - Trial data/15k_c_sample_quotes.csv")

# 3 versions of 20k samples:
corpi_20_a = create_corpus("../data/2 - Trial data/20k_a_sample_quotes.csv")
corpi_20_b = create_corpus("../data/2 - Trial data/20k_b_sample_quotes.csv")
corpi_20_c = create_corpus("../data/2 - Trial data/20k_c_sample_quotes.csv")

# 3 versions of 25k samples:
corpi_25_a = create_corpus("../data/2 - Trial data/25k_a_sample_quotes.csv")
corpi_25_b = create_corpus("../data/2 - Trial data/25k_b_sample_quotes.csv")
corpi_25_c = create_corpus("../data/2 - Trial data/25k_c_sample_quotes.csv")

# Topic modeling methods
In this part we will use topic modeling methods to find the most relevant topics in the data.


## LDA

#### Trial 1

In [3]:
trial_num = 1
corpus, dic, bow_corpus = corpi_25_a
topic_num = 25

lda_model = create_lda_model(dic, bow_corpus, topic_num, trial_num)
lda_topic_dict = create_lda_topic_dict(lda_model, topic_num)

Topic 0: ('0.008*"מהיר" + 0.007*"הצעות" + 0.007*"הצעה" + 0.007*"לסדר" + 0.007*"מיקי" + 0.006*"המקומיות" + 0.006*"הפנים" + 0.006*"לוי" + 0.006*"בקשה" + 0.005*"יושב-ראש"',)

Topic 1: ('0.005*"דיונים" + 0.004*"אתן" + 0.003*"אתנו" + 0.003*"זכות" + 0.003*"בפתח" + 0.003*"מיכאל" + 0.003*"להודות" + 0.003*"אלינו" + 0.003*"מהיר" + 0.003*"אילן"',)

Topic 2: ('0.010*"התחבורה" + 0.009*"ירושלים" + 0.008*"האוצר" + 0.007*"העלייה" + 0.006*"הציבורית" + 0.006*"הלאומי" + 0.005*"העיר" + 0.005*"הביטוח" + 0.004*"לאומי" + 0.004*"עיריית"',)

Topic 3: ('0.012*"שקלים" + 0.010*"מיליון" + 0.007*"שקל" + 0.005*"השואה" + 0.004*"בישיבה" + 0.004*"פי" + 0.004*"ככל" + 0.004*"מיליארד" + 0.004*"הציבור" + 0.004*"בישראל"',)

Topic 4: ('0.006*"המים" + 0.004*"ושומרון" + 0.003*"השיר" + 0.003*"ביהודה" + 0.003*"תאגידי" + 0.003*"חס" + 0.003*"טיפול" + 0.003*"וחלילה" + 0.002*"אדמה" + 0.002*"נקרא"',)

Topic 5: ('0.006*"התקנות" + 0.004*"בקשה" + 0.004*"להגיש" + 0.004*"המועצה" + 0.004*"הרביזיה" + 0.004*"רביזיה" + 0.004*"מחדש" + 0.004*"ה

In [5]:
manual_topic_dict = {11: "חינוך", 16: "נשים", 19: "בטחון", 15: "קורונה", 24: "קורונה"}
tag_df = create_tag_df(lda_model, bow_corpus, corpus, manual_topic_dict, lda_topic_dict, trial_num)
print(len(tag_df))
tag_df.head(50)

2299


,NonStopwordsQuoteText,Topic,Topic_Prob,TopicWords,Batch
2380,ברשותכם פותחת מיגור תופעת שכחת ברכב כסיכום רוו...,קורונה,0.983050,"(0.011*""הבריאות"" + 0.006*""החולים"" + 0.005*""הדב...",1
8954,חייב לציין ושיתפתי חברים והדברים נאמרו חוזר על...,קורונה,0.981380,"(0.011*""הבריאות"" + 0.006*""החולים"" + 0.005*""הדב...",1
267,בתקופת שיא וחשש לקראת החמרה במצב נצליח לצלוח ה...,קורונה,0.980242,"(0.011*""הבריאות"" + 0.006*""החולים"" + 0.005*""הדב...",1
105,שרשרת ההדבקה ברורה הסוף קיימת הערכה שאחד העובד...,קורונה,0.978508,"(0.011*""הבריאות"" + 0.006*""החולים"" + 0.005*""הדב...",1
6818,מודה השיתוף מבקשת ומעודדת להעלות כלפינו הידע ש...,קורונה,0.977560,"(0.011*""הבריאות"" + 0.006*""החולים"" + 0.005*""הדב...",1
11569,רב החוקה ומשפט ארבעה נושאים הכרזת סמכויות מיוח...,קורונה,0.975999,"(0.018*""הפנים"" + 0.018*""הקורונה"" + 0.017*""הורא...",1
7483,ובוקר שמי עמיחי שיקלי מפקד פלוגת צנחנים במילוא...,קורונה,0.975742,"(0.011*""הבריאות"" + 0.006*""החולים"" + 0.005*""הדב...",1
2956,להשתתף בצערם משפחות השוטרים האיל סתאווי וכמיל ...,קורונה,0.972759,"(0.011*""הבריאות"" + 0.006*""החולים"" + 0.005*""הדב...",1
12466,הבוקר נדון בתקנות הכרזת סמכויות מיוחדות להתמוד...,קורונה,0.972568,"(0.018*""הפנים"" + 0.018*""הקורונה"" + 0.017*""הורא...",1
4595,ברשותכם הבוקר להעלות זכרו הרב אהרון לייב שטיינ...,קורונה,0.972158,"(0.011*""הבריאות"" + 0.006*""החולים"" + 0.005*""הדב...",1


#### Trial 2

In [6]:
trial_num = 2
corpus, dic, bow_corpus = corpi_25_a
topic_num = 35

lda_model = create_lda_model(dic, bow_corpus, topic_num, trial_num)
lda_topic_dict = create_lda_topic_dict(lda_model, topic_num)

Topic 0: ('0.017*"מעמד" + 0.017*"לקידום" + 0.017*"האישה" + 0.010*"מגדרי" + 0.009*"שירותים" + 0.008*"פותחת" + 0.008*"ולשוויון" + 0.007*"הנוסח" + 0.007*"נוסח" + 0.007*"שינויים"',)

Topic 1: ('0.050*"הפנים" + 0.041*"הסביבה" + 0.027*"והגנת" + 0.007*"להגנת" + 0.005*"המקומיות" + 0.005*"עסקים" + 0.005*"טלי" + 0.004*"/" + 0.004*"פלוסקוב" + 0.004*"משוחררים"',)

Topic 2: ('0.006*"אתנו" + 0.005*"להודות" + 0.005*"הדברים" + 0.004*"שמחים" + 0.004*"הציבורי" + 0.004*"והיא" + 0.003*"לברכה" + 0.003*"לידי" + 0.003*"להתחיל" + 0.003*"טל"',)

Topic 3: ('0.007*"הפשיעה" + 0.005*"המידע" + 0.005*"בחברה" + 0.005*"בודדים" + 0.005*"מדינת" + 0.004*"יהודי" + 0.004*"הערבית" + 0.004*"הנחה" + 0.004*"הממשלתית" + 0.003*"חופש"',)

Topic 4: ('0.011*"איתנו" + 0.008*"נתחיל" + 0.008*"ירושלים" + 0.007*"המשפטי" + 0.007*"פקודת" + 0.007*"לתיקון" + 0.007*"מכן" + 0.006*"היועץ" + 0.006*"אתן" + 0.006*"המשפטים"',)

Topic 5: ('0.011*"השקיפות" + 0.011*"מידע" + 0.009*"שקיפות" + 0.007*"המשרד" + 0.005*"לציבור" + 0.005*"המידע" + 0.005*"אנרג

In [8]:
manual_topic_dict = {0: "נשים", 6: "ניצולי שואה", 19: "קורונה", 22: "כלכלי", 23: "חינוך", 29: "בריאות", 30: "כלכלי"}
tag_df = create_tag_df(lda_model, bow_corpus, corpus, manual_topic_dict, lda_topic_dict, trial_num)
print(len(tag_df))
tag_df.head(50)

1409


,NonStopwordsQuoteText,Topic,Topic_Prob,TopicWords,Batch
8305,תשמע נתנו לבנות עשרים ושלוש גנים באתי לבנות ופ...,בריאות,0.984275,"(0.016*""החולים"" + 0.013*""הבריאות"" + 0.010*""חול...",2
8621,והחלק למליאה יידון החלקים שעוברים למליאה הדו-ש...,קורונה,0.981657,"(0.021*""התקנות"" + 0.020*""הקורונה"" + 0.012*""הור...",2
6880,ב-4 במאי טטיאנה חייקין 50 במותה נרצחה פי החשד ...,חינוך,0.975592,"(0.091*""החינוך"" + 0.024*""התרבות"" + 0.022*""והספ...",2
6673,צוהרים החוקה סמכויות מיוחדות להתמודדות נגיף הק...,קורונה,0.971373,"(0.021*""התקנות"" + 0.020*""הקורונה"" + 0.012*""הור...",2
8436,אלינו חברה שהגיעו מתגלית שאותם נציג לעומק מאוח...,בריאות,0.971321,"(0.016*""החולים"" + 0.013*""הבריאות"" + 0.010*""חול...",2
7887,פותחים כלכלה בסוגיה כואבת במדינה הקטל בדרכים מ...,בריאות,0.970463,"(0.016*""החולים"" + 0.013*""הבריאות"" + 0.010*""חול...",2
9703,שמתקשר אלינו במהלך שיחה אובדנית למעלה משניים י...,בריאות,0.969587,"(0.016*""החולים"" + 0.013*""הבריאות"" + 0.010*""חול...",2
9957,העניין שנמצאים סביב השולחן וכולם צודקים פלטפור...,ניצולי שואה,0.969559,"(0.008*""האוצר"" + 0.005*""החלטה"" + 0.005*""לקיים""...",2
436,אחזור התהליך כולו שמאותה ידיעה שנאמרה במצרים ל...,ניצולי שואה,0.969460,"(0.008*""האוצר"" + 0.005*""החלטה"" + 0.005*""לקיים""...",2
9544,ניסחנו ההסכמות שהוצגו עקרוני בישיבות הקודמות ו...,נשים,0.968616,"(0.017*""מעמד"" + 0.017*""לקידום"" + 0.017*""האישה""...",2


#### Trial 3

In [9]:
trial_num = 3
corpus, dic, bow_corpus = corpi_25_b
topic_num = 50

lda_model = create_lda_model(dic, bow_corpus, topic_num, trial_num)
lda_topic_dict = create_lda_topic_dict(lda_model, topic_num)

Topic 0: ('0.011*"אתיופיה" + 0.008*"יוצאי" + 0.007*"הנוער" + 0.007*"נוער" + 0.007*"הארץ" + 0.006*"פערים" + 0.006*"הגבוהה" + 0.005*"הסטודנטים" + 0.005*"ההשכלה" + 0.005*"הפריפריה"',)

Topic 1: ('0.013*"גבוהה" + 0.007*"להשכלה" + 0.006*"בתחום" + 0.006*"אילן" + 0.005*"המשרד" + 0.005*"מחקר" + 0.004*"הבאה" + 0.004*"החינוך" + 0.004*"ברמה" + 0.004*"רבים"',)

Topic 2: ('0.009*"אישור" + 0.008*"דיבור" + 0.005*"מבין" + 0.005*"מיוחדים" + 0.005*"השכר" + 0.005*"המשפטיים" + 0.005*"העירייה" + 0.005*"צרכים" + 0.004*"הכספים" + 0.004*"הטבות"',)

Topic 3: ('0.024*"מכן" + 0.020*"נתחיל" + 0.013*"ולאחר" + 0.012*"אתן" + 0.011*"הדיבור" + 0.010*"נעבור" + 0.010*"הדברים" + 0.009*"ואחר" + 0.007*"הערות" + 0.006*"זכות"',)

Topic 4: ('0.006*"ושומרון" + 0.005*"ביהודה" + 0.004*"מחסור" + 0.004*"בחירות" + 0.003*"שקל" + 0.003*"העניין" + 0.003*"גבייה" + 0.003*"לשנה" + 0.003*"חברות" + 0.003*"והם"',)

Topic 5: ('0.008*"התעופה" + 0.007*"בתי" + 0.006*"המשטרה" + 0.006*"חודשים" + 0.005*"שדה" + 0.005*"המים" + 0.005*"להאריך" + 0.005

In [10]:
manual_topic_dict = {0: "עולים", 1: "חינוך", 8: "חינוך", 10: "בטחון", 16: "קורונה", 18: "תחבורה", 19: "בטחון", 20: "עולים", 28: "בריאות", 29: "נשים", 34: "קורונה", 38: "עולים", 39: "כלכלי", 40: "דיור", 46: "נשים"}
tag_df = create_tag_df(lda_model, bow_corpus, corpus, manual_topic_dict, lda_topic_dict, trial_num)
print(len(tag_df))
tag_df.head(50)

1947


,NonStopwordsQuoteText,Topic,Topic_Prob,TopicWords,Batch
8858,מתקבלות במיטב 6600 פניות מלשבים כמות עצומה המע...,עולים,0.987733,"(0.011*""אתיופיה"" + 0.008*""יוצאי"" + 0.007*""הנוע...",3
1458,הנסבות המחמירות כוללות מקרי הרצח החמורים שמצדי...,עולים,0.987393,"(0.027*""העלייה"" + 0.018*""הקליטה"" + 0.014*""מנכל...",3
2007,הבריאות סירב תוקף למתווה שאומר שהישיבה בחוץ פי...,קורונה,0.987091,"(0.017*""הקורונה"" + 0.012*""משבר"" + 0.006*""צהל"" ...",3
806,משנות ה-50 עדים לאפליה רקע אמונה דתית והרקע לא...,תחבורה,0.986546,"(0.024*""התחבורה"" + 0.012*""הציבורית"" + 0.006*""ב...",3
1054,מאגף התקציבים באוצר נציג השינוי מבנה הבעלות בש...,כלכלי,0.985337,"(0.013*""האוצר"" + 0.008*""הכנסה"" + 0.007*""עסקים""...",3
6739,בשיחות שקיימתי האוצר ובשיחות שקיימתי יושב-ראש ...,בטחון,0.984901,"(0.007*""חיילים"" + 0.004*""וביטחון"" + 0.004*""ועד...",3
9140,בהקשר הנכון היושב-ראש כיוון שהביקורת הסגירה הס...,בטחון,0.984432,"(0.007*""חיילים"" + 0.004*""וביטחון"" + 0.004*""ועד...",3
1532,החוקה חמישה סטים במגוון נושאים הנושאים ההגבלות...,קורונה,0.984431,"(0.037*""הקורונה"" + 0.029*""שקלים"" + 0.024*""הורא...",3
8519,תקנת השבים נתת בהלכה היהודית מניח במערכת כולה ...,בריאות,0.984427,"(0.032*""הבריאות"" + 0.028*""החולים"" + 0.017*""חול...",3
9163,בגדול בסך הכול אתרים מקודשים לעם היהודי לעם הי...,בטחון,0.984175,"(0.007*""חיילים"" + 0.004*""וביטחון"" + 0.004*""ועד...",3


#### Trial 4

In [11]:
trial_num = 4
corpus, dic, bow_corpus = corpi_25_b
topic_num = 35

lda_model = create_lda_model(dic, bow_corpus, topic_num, trial_num)
lda_topic_dict = create_lda_topic_dict(lda_model, topic_num)

Topic 0: ('0.013*"הבאים" + 0.013*"יושב" + 0.012*"העלייה" + 0.011*"ברוכים" + 0.009*"מר" + 0.009*"אלי" + 0.009*"הקליטה" + 0.009*"כהן" + 0.009*"איתנו" + 0.007*"מאיר"',)

Topic 1: ('0.012*"ככל" + 0.009*"הדברים" + 0.008*"הציבור" + 0.006*"לפרוטוקול" + 0.005*"קטינים" + 0.005*"דוברים" + 0.005*"תקופת" + 0.004*"בחירות" + 0.004*"לתקנון" + 0.004*"בקשה"',)

Topic 2: ('0.023*"הקורונה" + 0.018*"הוראת" + 0.014*"העבודה" + 0.012*"נגיף" + 0.011*"הרווחה" + 0.009*"והבריאות" + 0.008*"סמכויות" + 0.007*"מיוחדות" + 0.007*"להתמודדות" + 0.007*"בדצמבר"',)

Topic 3: ('0.005*"הכספים" + 0.004*"אמרתי" + 0.004*"משנה" + 0.004*"החברות" + 0.003*"ועדה" + 0.003*"לאשר" + 0.003*"הכנסה" + 0.003*"כלל" + 0.003*"להקים" + 0.003*"החורף"',)

Topic 4: ('0.017*"החולים" + 0.017*"הבריאות" + 0.010*"חולים" + 0.006*"החלטה" + 0.005*"החלטת" + 0.005*"הביטחון" + 0.005*"החלטות" + 0.004*"בתי" + 0.004*"ההסדר" + 0.004*"קופות"',)

Topic 5: ('0.006*"בתי" + 0.006*"לאומי" + 0.005*"צריכים" + 0.004*"בבתי" + 0.004*"המוסד" + 0.004*"לביטוח" + 0.004*"אנשי"

In [12]:
manual_topic_dict = {2: "קורונה", 4: "בריאות", 8: "דיור", 11: "חינוך", 12: "התנחלויות", 13: "תחבורה", 16: "בטחון", 17: "עולים", 20: "נשים", 21: "נשים", 26: "כלכלי"}
tag_df = create_tag_df(lda_model, bow_corpus, corpus, manual_topic_dict, lda_topic_dict, trial_num)
print(len(tag_df))
tag_df.head(50)

2097


,NonStopwordsQuoteText,Topic,Topic_Prob,TopicWords,Batch
1322,העבודה והרווחה והשירותים החברתיים יישם תוכנית ...,נשים,0.984304,"(0.021*""נשים"" + 0.011*""אלימות"" + 0.007*""מינית""...",4
254,המשימה השנייה שיתופי פעולה עלינו להכיר מקבילינ...,עולים,0.982528,"(0.011*""עולים"" + 0.010*""בישראל"" + 0.008*""חדשים...",4
8386,מכובדיי צוהריים שממשיך אתנו מהדיון הקודם שהצטר...,קורונה,0.980928,"(0.023*""הקורונה"" + 0.018*""הוראת"" + 0.014*""העבו...",4
161,מסתכלים ההצדקה נקודת המוצא קורא חובת ההגנה העצ...,כלכלי,0.980067,"(0.019*""שקלים"" + 0.015*""מיליון"" + 0.010*""שקל"" ...",4
2044,פורסמו מחקרים ותמיד ויכוח קליקה סגורה שכולם נר...,התנחלויות,0.976717,"(0.006*""ושומרון"" + 0.005*""צריכים"" + 0.004*""נוש...",4
18822,רב החוקה ומשפט ארבעה נושאים הכרזת סמכויות מיוח...,קורונה,0.976304,"(0.023*""הקורונה"" + 0.018*""הוראת"" + 0.014*""העבו...",4
1525,להתחיל אתך פרופסור גרוטו וזאת הזדמנות מצוינת ל...,נשים,0.976227,"(0.013*""לקידום"" + 0.013*""מעמד"" + 0.012*""האישה""...",4
4524,שיגרתי מכתבים לשני גדולי תורה מכבד ומוקיר מעומ...,בטחון,0.975584,"(0.005*""האוצר"" + 0.004*""חיילים"" + 0.004*""תקציב...",4
5596,שאפשר להתייחס אליו בתור הדייר הסרבן ואיך מקטינ...,נשים,0.973669,"(0.021*""נשים"" + 0.011*""אלימות"" + 0.007*""מינית""...",4
1008,מעלה השאלה נדרש נעשה שימוש וכמה מעלה השאלות פת...,תחבורה,0.972924,"(0.009*""ציבורית"" + 0.007*""הממלכה"" + 0.007*""התח...",4


#### Trial 5

In [13]:
trial_num = 5
corpus, dic, bow_corpus = corpi_20_a
topic_num = 30

lda_model = create_lda_model(dic, bow_corpus, topic_num, trial_num)
lda_topic_dict = create_lda_topic_dict(lda_model, topic_num)

Topic 0: ('0.008*"הכנסה" + 0.004*"הבטחת" + 0.003*"שינוי" + 0.003*"העדה" + 0.003*"בזק" + 0.003*"הישראלית" + 0.002*"התקשורת" + 0.002*"צריכים" + 0.002*"נושאים" + 0.002*"בחברה"',)

Topic 1: ('0.011*"הממלכה" + 0.009*"לאנשי" + 0.009*"הדין" + 0.007*"עורכי" + 0.006*"אנשי" + 0.005*"לשכת" + 0.005*"דין" + 0.004*"הפלילי" + 0.004*"לחברי" + 0.003*"בתי"',)

Topic 2: ('0.019*"שקלים" + 0.016*"מיליון" + 0.011*"שקל" + 0.006*"החינוך" + 0.006*"מיליארד" + 0.005*"שנת" + 0.004*"הערבי" + 0.004*"במגזר" + 0.004*"הלימודים" + 0.003*"המקומית"',)

Topic 3: ('0.007*"הבריאות" + 0.006*"הדברים" + 0.005*"האוצר" + 0.003*"למשרד" + 0.003*"המצב" + 0.003*"להבין" + 0.003*"מצד" + 0.003*"המשטרה" + 0.003*"תקציב" + 0.003*"השר"',)

Topic 4: ('0.006*"מוסדות" + 0.006*"הספורט" + 0.004*"ציבור" + 0.004*"רשימה" + 0.004*"הכספים" + 0.004*"אישור" + 0.003*"46" + 0.003*"הביטחון" + 0.003*"לפקודת" + 0.003*"מקרקעין"',)

Topic 5: ('0.008*"ערך" + 0.005*"הבריאות" + 0.004*"הדברים" + 0.004*"הרשות" + 0.004*"ניירות" + 0.003*"המחיה" + 0.003*"יוקר" + 0.0

In [14]:
manual_topic_dict = {5: "כלכלי", 6: "נשים", 12: "עולים", 14: "קורונה", 15: "בריאות", 17: "נשים", 18: "רווחה",26:"חינוך",29:"בריאות"}
tag_df = create_tag_df(lda_model, bow_corpus, corpus, manual_topic_dict, lda_topic_dict, trial_num)
print(len(tag_df))
tag_df.head(50)

2340


,NonStopwordsQuoteText,Topic,Topic_Prob,TopicWords,Batch
5365,במרוצת וביתר בחודשים האחרונים נחשפות בזו עדויו...,בריאות,0.991104,"(0.009*""החולים"" + 0.006*""בתי"" + 0.004*""הבריאות...",5
5401,ברשותכם אפתח הבוקר בשיר מילים לזכרו הרב אהרון ...,עולים,0.990128,"(0.013*""העלייה"" + 0.009*""הקליטה"" + 0.008*""הבאי...",5
5035,נוהל פונדקאות חול שעליו ניהלנו בשנה שעברה שנתי...,בריאות,0.988334,"(0.008*""מבקר"" + 0.003*""הבריאות"" + 0.003*""אומרת...",5
9591,שישי קיבלנו הוכחה מצמררת רב סמל אסף אריה סגן מ...,בריאות,0.988039,"(0.009*""החולים"" + 0.006*""בתי"" + 0.004*""הבריאות...",5
1660,השורש הסוגיה עליה נובע שהחינוך העל יסודי בישרא...,חינוך,0.987715,"(0.063*""החינוך"" + 0.018*""התרבות"" + 0.015*""והספ...",5
4710,שנתחיל לציין שניתנה שמענו בתקשורת לקראת חגיגות...,נשים,0.987086,"(0.006*""נשים"" + 0.005*""המקומי"" + 0.004*""השלטון...",5
4569,מאחר ואמרתי להעיר תשומת לבכם לחידוש בחדר התחלנ...,כלכלי,0.986730,"(0.008*""ערך"" + 0.005*""הבריאות"" + 0.004*""הדברים...",5
5815,הצטרפתי לממשלה האמנתי שאפשר ביחד לעם קיוויתי ש...,נשים,0.986723,"(0.017*""האישה"" + 0.015*""מעמד"" + 0.015*""לקידום""...",5
2732,ההיי-טק הישראלי אתגרים מאופיין בכך מתחום שההשת...,נשים,0.986154,"(0.006*""נשים"" + 0.005*""המקומי"" + 0.004*""השלטון...",5
5371,החוקה ארבעה נושאים באותו עניין הסדר הכרונולוגי...,קורונה,0.985776,"(0.036*""הקורונה"" + 0.026*""הוראת"" + 0.020*""נגיף...",5


In [15]:
trial_num = 6
corpus, dic, bow_corpus = corpi_20_a
topic_num = 40

lda_model = create_lda_model(dic, bow_corpus, topic_num, trial_num)
lda_topic_dict = create_lda_topic_dict(lda_model, topic_num)

Topic 0: ('0.005*"נכנס" + 0.004*"לתוקף" + 0.004*"התקנות" + 0.003*"קטי" + 0.003*"שהחוק" + 0.003*"ההמלצות" + 0.003*"השר" + 0.003*"ילדי" + 0.003*"שטרית" + 0.003*"תימן"',)

Topic 1: ('0.024*"נשים" + 0.007*"ציבור" + 0.006*"אלימות" + 0.006*"הכנסה" + 0.005*"מוסדות" + 0.005*"הנשים" + 0.004*"לנשים" + 0.003*"חייבת" + 0.003*"במשפחה" + 0.003*"רשימה"',)

Topic 2: ('0.006*"הדברים" + 0.004*"יכולים" + 0.004*"להבין" + 0.004*"שצריך" + 0.003*"בסופו" + 0.003*"מדינת" + 0.003*"בישראל" + 0.003*"הקורונה" + 0.003*"צריכים" + 0.003*"אומרת"',)

Topic 3: ('0.022*"שקלים" + 0.016*"מיליון" + 0.009*"שקל" + 0.007*"מיליארד" + 0.005*"מבקר" + 0.004*"ושומרון" + 0.003*"יהודה" + 0.003*"קו" + 0.003*"ביהודה" + 0.003*"מפעל"',)

Topic 4: ('0.007*"החינוך" + 0.007*"גבוהה" + 0.006*"הגבוהה" + 0.006*"להשכלה" + 0.004*"ההשכלה" + 0.004*"במוסדות" + 0.004*"מודה" + 0.004*"התרבות" + 0.004*"החשמל" + 0.004*"האנרגיה"',)

Topic 5: ('0.006*"חברים" + 0.006*"עודד" + 0.006*"ביטון" + 0.005*"פורר" + 0.005*"פלוסקוב" + 0.005*"טלי" + 0.004*"יפעת" + 0.00

In [16]:
manual_topic_dict = {1: "נשים", 4: "חינוך", 12: "עולים", 13: "חינוך", 15: "בריאות", 17: "כלכלי", 18: "בריאות", 25: "כלכלי", 27: "תחבורה", 34: "קורונה", 35: "נשים"}
tag_df = create_tag_df(lda_model, bow_corpus, corpus, manual_topic_dict, lda_topic_dict, trial_num)
print(len(tag_df))
tag_df.head(50)

2037


,NonStopwordsQuoteText,Topic,Topic_Prob,TopicWords,Batch
1102,בסקר עדכני שקיימו בשבוע איגוד הארגונים היהודיי...,חינוך,0.989718,"(0.007*""החינוך"" + 0.007*""גבוהה"" + 0.006*""הגבוה...",6
5035,נוהל פונדקאות חול שעליו ניהלנו בשנה שעברה שנתי...,בריאות,0.988240,"(0.022*""החולים"" + 0.022*""בתי"" + 0.017*""הספר"" +...",6
521,לכולן ולכולם לחברת רוזין לחבר בגין נוסף חיסיון...,תחבורה,0.988096,"(0.018*""התחבורה"" + 0.017*""הדין"" + 0.016*""המדע""...",6
6585,בטרם קריסת חברה קרסה חברה שאמרתי היות ואסור לע...,נשים,0.987472,"(0.024*""נשים"" + 0.007*""ציבור"" + 0.006*""אלימות""...",6
1418,באחד מהביקורים בשניים שלושה לשכות רווחה הנושאי...,בריאות,0.987139,"(0.022*""החולים"" + 0.022*""בתי"" + 0.017*""הספר"" +...",6
9503,אוקיי הבעיה שבו היתר הפקודה רישיון ממנכל הבריא...,עולים,0.986624,"(0.016*""העלייה"" + 0.015*""עולים"" + 0.013*""הקליט...",6
5617,בשונה מהמצב שנהוג בחלק ממדינות בישראל חובת שקי...,נשים,0.986238,"(0.024*""נשים"" + 0.007*""ציבור"" + 0.006*""אלימות""...",6
9378,הזיהום פוגע במקורות המים הזמינים באמינות האספק...,חינוך,0.986050,"(0.007*""החינוך"" + 0.007*""גבוהה"" + 0.006*""הגבוה...",6
2732,ההיי-טק הישראלי אתגרים מאופיין בכך מתחום שההשת...,נשים,0.986018,"(0.016*""מעמד"" + 0.015*""האישה"" + 0.014*""לקידום""...",6
5371,החוקה ארבעה נושאים באותו עניין הסדר הכרונולוגי...,קורונה,0.985654,"(0.029*""הקורונה"" + 0.021*""הוראת"" + 0.014*""נגיף...",6


In [17]:
trial_num = 7
corpus, dic, bow_corpus = corpi_20_b
topic_num = 40

lda_model = create_lda_model(dic, bow_corpus, topic_num, trial_num)
lda_topic_dict = create_lda_topic_dict(lda_model, topic_num)

Topic 0: ('0.008*"השירות" + 0.005*"הקודם" + 0.004*"הקודמת" + 0.004*"הדירה" + 0.003*"בישיבה" + 0.003*"הפנייה" + 0.003*"נקודה" + 0.003*"חובה" + 0.002*"אחוזים" + 0.002*"גדולים"',)

Topic 1: ('0.005*"בתחום" + 0.004*"ושומרון" + 0.004*"רפואי" + 0.004*"טיפול" + 0.004*"הסמים" + 0.003*"הטיפול" + 0.003*"ביהודה" + 0.003*"לטיפול" + 0.003*"שצריך" + 0.003*"חייבת"',)

Topic 2: ('0.004*"הדברים" + 0.003*"צריכים" + 0.003*"הביטחון" + 0.003*"המצב" + 0.003*"ערך" + 0.003*"חייב" + 0.002*"הקודמת" + 0.002*"עובדים" + 0.002*"יכולים" + 0.002*"אותנו"',)

Topic 3: ('0.010*"העולים" + 0.009*"עולים" + 0.008*"אתיופיה" + 0.007*"יוצאי" + 0.007*"חיילים" + 0.006*"החדשים" + 0.005*"בישראל" + 0.004*"בודדים" + 0.003*"משוחררים" + 0.003*"השכלה"',)

Topic 4: ('0.013*"הדין" + 0.009*"איתנו" + 0.009*"כהן" + 0.008*"אבו" + 0.008*"מאיר" + 0.007*"עורכת" + 0.006*"רועי" + 0.006*"פולקמן" + 0.005*"ארי" + 0.005*"מירב"',)

Topic 5: ('0.019*"תקציב" + 0.006*"הכספים" + 0.005*"האוצר" + 0.004*"המצב" + 0.004*"המחיה" + 0.003*"התקציב" + 0.003*"באפריל

In [18]:
manual_topic_dict = {2: "בטחון", 3: "עולים", 5: "כלכלי", 9: "בטחון", 11: "קורונה", 12: "חינוך", 17: "עולים", 18:"בריאות", 23: "נשים", 36: "ניצולי שואה"}
tag_df = create_tag_df(lda_model, bow_corpus, corpus, manual_topic_dict, lda_topic_dict, trial_num)
print(len(tag_df))
tag_df.head(50)

1967


,NonStopwordsQuoteText,Topic,Topic_Prob,TopicWords,Batch
9948,הרקע לקביעת עניינים שונים קשורים קיבלנו פניות ...,ניצולי שואה,0.988907,"(0.008*""השואה"" + 0.005*""ניצולי"" + 0.005*""בשבת""...",7
9010,הנתונים מקבל כ-30000 שעובדים בירושלים המזרחית ...,חינוך,0.988089,"(0.069*""החינוך"" + 0.020*""התרבות"" + 0.017*""והספ...",7
6289,חייבת שמדינת וזאת הסיבה שלקחנו בפרספקטיבה 30 3...,ניצולי שואה,0.987146,"(0.008*""השואה"" + 0.005*""ניצולי"" + 0.005*""בשבת""...",7
7023,לנוכח המלחמה הארורה והקשה ועצובה חייבת לשתף שב...,ניצולי שואה,0.986791,"(0.008*""השואה"" + 0.005*""ניצולי"" + 0.005*""בשבת""...",7
1822,במלאת כעשרים ויותר לחוק הטרדות מיניות חשבתי שר...,חינוך,0.986787,"(0.069*""החינוך"" + 0.020*""התרבות"" + 0.017*""והספ...",7
9942,כול מזדהה שאמרת שהמטרה להימנע מלהגיע להליכים מ...,כלכלי,0.986441,"(0.019*""תקציב"" + 0.006*""הכספים"" + 0.005*""האוצר...",7
1928,פונה שפניתי בישיבה הקודמת פניתי אישית לשר במת ...,עולים,0.985842,"(0.010*""העולים"" + 0.009*""עולים"" + 0.008*""אתיופ...",7
2914,שהתגלה בדוח חייבת שאמנם נפלתי מהכיסא מתואר בעי...,כלכלי,0.985425,"(0.019*""תקציב"" + 0.006*""הכספים"" + 0.005*""האוצר...",7
3425,בסעיף האחרון בסדר-היום סכנת סגירת המאפייה המרח...,עולים,0.984736,"(0.010*""העולים"" + 0.009*""עולים"" + 0.008*""אתיופ...",7
1593,מאז שרוי בערפל קרה ידיעות שיותר מ-10% מהבדיקות...,בריאות,0.984734,"(0.021*""מבקר"" + 0.014*""החולים"" + 0.007*""הבריאו...",7


In [19]:
trial_num = 8
corpus, dic, bow_corpus = corpi_20_c
topic_num = 40

lda_model = create_lda_model(dic, bow_corpus, topic_num, trial_num)
lda_topic_dict = create_lda_topic_dict(lda_model, topic_num)

Topic 0: ('0.015*"האוצר" + 0.011*"הבריאות" + 0.007*"החולים" + 0.007*"ממשרד" + 0.004*"תקציב" + 0.004*"/" + 0.004*"מרכזי" + 0.004*"הספורט" + 0.004*"קיבלנו" + 0.003*"חולים"',)

Topic 1: ('0.003*"לוועדה" + 0.003*"הטבות" + 0.003*"המשפטית" + 0.003*"אתר" + 0.002*"מיני" + 0.002*"מבין" + 0.002*"להכיר" + 0.002*"שבו" + 0.002*"מבחינתנו" + 0.002*"הפעם"',)

Topic 2: ('0.006*"עאידה" + 0.006*"תומא" + 0.006*"עבירות" + 0.005*"הצעה" + 0.005*"דירה" + 0.005*"סלימאן" + 0.005*"הצעות" + 0.004*"שתי" + 0.004*"שירות" + 0.004*"ראובן"',)

Topic 3: ('0.008*"התחבורה" + 0.007*"הציבורית" + 0.005*"הדברים" + 0.004*"המים" + 0.004*"ציבורית" + 0.004*"מכן" + 0.004*"לדון" + 0.004*"האוצר" + 0.004*"הבריאות" + 0.003*"הכספים"',)

Topic 4: ('0.012*"בדרכים" + 0.005*"פנים" + 0.004*"לביטחון" + 0.004*"לבטיחות" + 0.003*"המצב" + 0.003*"המשטרה" + 0.003*"הקטל" + 0.003*"הכיסאות" + 0.002*"תשעט" + 0.002*"בקטל"',)

Topic 5: ('0.011*"מהיר" + 0.011*"לוי" + 0.009*"מיקי" + 0.007*"יושב-ראש" + 0.007*"הצעה" + 0.006*"כהן" + 0.005*"מירב" + 0.005*"ארי

In [20]:
manual_topic_dict = {3: "תחבורה", 4: "תחבורה", 7: "דיור", 8: "חינוך", 16: "קורונה", 21: "כלכלי", 23: "רווחה", 25: "תחבורה",27: "חינוך", 29: "בריאות", 32: "נשים", 38: "ניצולי שואה"}
tag_df = create_tag_df(lda_model, bow_corpus, corpus, manual_topic_dict, lda_topic_dict, trial_num)
print(len(tag_df))
tag_df.head(50)

2360


,NonStopwordsQuoteText,Topic,Topic_Prob,TopicWords,Batch
7265,ברשותכם הבוקר להעלות זכרו הרב אהרון לייב שטיינ...,חינוך,0.990242,"(0.006*""גבוהה"" + 0.006*""הכנסה"" + 0.005*""אישור""...",8
916,6 לחוק נישואין בהצעה שהתחילה ה-18 על-ידי צירוף...,בריאות,0.989716,"(0.011*""בתי"" + 0.011*""החולים"" + 0.006*""חולים"" ...",8
6957,ראשית מברכת היושבת שקיים חייבת לציין בזכותה גי...,בריאות,0.987941,"(0.011*""בתי"" + 0.011*""החולים"" + 0.006*""חולים"" ...",8
8280,הספיק להתעדכן בסיור הקבינט בסיכום הנחה המועצה ...,רווחה,0.986621,"(0.012*""הילד"" + 0.005*""הבריאות"" + 0.005*""אומרת...",8
8693,ביקשנו לזמן מודה שבאתם אמנם עברו שבועות מאז הש...,חינוך,0.986612,"(0.080*""החינוך"" + 0.023*""התרבות"" + 0.020*""והספ...",8
5000,מעבר למנוע אסון לאומי להתגייס למנוע אסון לאומי...,נשים,0.986612,"(0.033*""נשים"" + 0.016*""מעמד"" + 0.016*""לקידום"" ...",8
8839,ביקשתם לאור אירוע שהתרחש כחודש בבריכת השחייה ב...,תחבורה,0.986248,"(0.008*""העובדים"" + 0.005*""הכול"" + 0.004*""הרוגי...",8
6876,הגיבורים הללו עזבו כפריהם עזבו ביתם ויצאו למסע...,דיור,0.986228,"(0.006*""הציבורי"" + 0.006*""הדיור"" + 0.006*""מדינ...",8
8239,עדות נוספת עדות מצולמת חיה שנעשו ניסויים באתר ...,תחבורה,0.986053,"(0.012*""בדרכים"" + 0.005*""פנים"" + 0.004*""לביטחו...",8
2950,המקרה גבעת עמל מתייחסים אליו כמקרה ייחודי דופן...,ניצולי שואה,0.985645,"(0.010*""השואה"" + 0.009*""הביטחון"" + 0.008*""ניצו...",8


# Creating a tagged dataframe
In this section, topics that were found in the previous section are used to create a final dataframe where each qoute is attached to a raw tag (topic that was discovered during the topic modeling phase, and was given an explicit topic name).
These raw topic tags will later be validated by human annotators, and after the validation will be used to train a topic classification model.

In [23]:
# create joined dataframe with tags from all trials
tagged_df = pd.DataFrame(columns=['NonStopwordsQuoteText', 'Topic', 'Topic_Prob', 'TopicWords'])
for trial in range(1,9):
    trial_df = pd.read_csv(f"../data/3 - Topic modeling trial data/tag_df_trial_{trial}.csv")
    tagged_df = pd.concat([tagged_df, trial_df])

# remove duplicates - if the same quote got more than 1 tag, keep the one with the hightest probabilty
tagged_df.sort_values(by=['Topic_Prob'], ascending=False, inplace=True)
print("before dedup:", len(tagged_df))
tagged_df.drop_duplicates(subset=['NonStopwordsQuoteText'], keep='first', inplace=True)
print("after dedup:", len(tagged_df))
tagged_df.drop(columns=['Unnamed: 0'], inplace=True)
tagged_df.to_csv(f"../data/3 - Topic modeling trial data/tag_df_all_trials.csv")
tagged_df.head(50)

before dedup: 16456
after dedup: 11766


,NonStopwordsQuoteText,Topic,Topic_Prob,TopicWords,Batch
0,במרוצת וביתר בחודשים האחרונים נחשפות בזו עדויו...,בריאות,0.991104,"('0.009*""החולים"" + 0.006*""בתי"" + 0.004*""הבריאו...",5.0
0,ברשותכם הבוקר להעלות זכרו הרב אהרון לייב שטיינ...,חינוך,0.990242,"('0.006*""גבוהה"" + 0.006*""הכנסה"" + 0.005*""אישור...",8.0
1,ברשותכם אפתח הבוקר בשיר מילים לזכרו הרב אהרון ...,עולים,0.990128,"('0.013*""העלייה"" + 0.009*""הקליטה"" + 0.008*""הבא...",5.0
0,בסקר עדכני שקיימו בשבוע איגוד הארגונים היהודיי...,חינוך,0.989718,"('0.007*""החינוך"" + 0.007*""גבוהה"" + 0.006*""הגבו...",6.0
1,6 לחוק נישואין בהצעה שהתחילה ה-18 על-ידי צירוף...,בריאות,0.989716,"('0.011*""בתי"" + 0.011*""החולים"" + 0.006*""חולים""...",8.0
0,הרקע לקביעת עניינים שונים קשורים קיבלנו פניות ...,ניצולי שואה,0.988907,"('0.008*""השואה"" + 0.005*""ניצולי"" + 0.005*""בשבת...",7.0
2,נוהל פונדקאות חול שעליו ניהלנו בשנה שעברה שנתי...,בריאות,0.988334,"('0.008*""מבקר"" + 0.003*""הבריאות"" + 0.003*""אומר...",5.0
2,לכולן ולכולם לחברת רוזין לחבר בגין נוסף חיסיון...,תחבורה,0.988096,"('0.018*""התחבורה"" + 0.017*""הדין"" + 0.016*""המדע...",6.0
1,הנתונים מקבל כ-30000 שעובדים בירושלים המזרחית ...,חינוך,0.988089,"('0.069*""החינוך"" + 0.020*""התרבות"" + 0.017*""והס...",7.0
3,שישי קיבלנו הוכחה מצמררת רב סמל אסף אריה סגן מ...,בריאות,0.988039,"('0.009*""החולים"" + 0.006*""בתי"" + 0.004*""הבריאו...",5.0


In [34]:
# join with original dataframe
main_df = pd.read_csv(f"../data/1 - Original data/PreprocessedData.csv")
final_tag_df = main_df.merge(right=tagged_df, how='left', on='NonStopwordsQuoteText')
final_tag_df = final_tag_df[~(final_tag_df['Topic'].isna()) & ~(final_tag_df['Batch'].isna())]
topics = final_tag_df.Topic.unique()
final_tag_df = final_tag_df[final_tag_df['Topic'].isin(topics)]
final_tag_df.fillna("missing", inplace=True)
final_tag_df.reset_index(drop=True, inplace=True)
print("final df size:",len(final_tag_df))
final_tag_df.to_csv(f"../data/4 - Auto tagged data/auto_tagged_data.csv", index=False)
final_tag_df.head(50)

final df size: 11925


,PersonID,Speaker,QuoteText,NonStopwordsQuoteText,QuoteWordCount,QuoteLen,StartDate,$Type,Topic,Topic_Prob,TopicWords,Batch
0,23632.0,מיקי לוי (יש עתיד):,"אדוני היושב-ראש, אדוני סגן שר החינוך, אני רוצה...",היושב-ראש סגן החינוך לשאול הובא לידיעתי השעיית...,42,243,2016-03-09 11:00:00,Plenum,כלכלי,0.763348,"('0.004*""הדברים"" + 0.004*""האוצר"" + 0.003*""גופי...",6.0
1,365.0,סגן מזכירת הכנסת נאזם בדר:,"אבקש מחברי הכנסת ומהקהל, האורחים, לכבד את יושב...",אבקש מחברי ומהקהל האורחים לכבד יושב-ראש הנשיא ...,16,94,2016-10-31 16:00:00,Plenum,נשים,0.703000,"('0.013*""לקידום"" + 0.013*""מעמד"" + 0.012*""האישה...",4.0
2,532.0,"היו""ר יולי יואל אדלשטיין:","אנחנו נתחיל עם שאילתות דחופות, אבל לפני השאילת...",נתחיל שאילתות דחופות השאילתות הדחופות חברות בר...,41,241,2016-11-30 11:00:00,Plenum,תחבורה,0.872175,"('0.008*""התחבורה"" + 0.007*""הציבורית"" + 0.005*""...",8.0
3,23566.0,סתיו שפיר (המחנה הציוני):,"תודה, כבוד היושב-ראש. על רקע הפגיעה המתמשכת בש...",היושב-ראש רקע הפגיעה המתמשכת בשכר המורים והמור...,28,162,2017-01-11 11:00:00,Plenum,קורונה,0.550844,"('0.036*""הקורונה"" + 0.026*""הוראת"" + 0.020*""נגי...",5.0
4,4355.0,יואל חסון (המחנה הציוני):,"אדוני היושב-ראש, אדוני סגן השר, בוקר טוב. השאל...",היושב-ראש סגן השר השאלה העומס החריג בפעילות סנ...,54,299,2017-06-14 11:00:00,Plenum,בריאות,0.975591,"('0.021*""מבקר"" + 0.014*""החולים"" + 0.007*""הבריא...",7.0
5,12938.0,"היו""ר חמד עמאר:",הנושא הראשון: שאילתות דחופות. אני מזמין את סגן...,שאילתות דחופות סגן החינוך מאיר פרוש להשיב שאיל...,41,208,2017-11-01 11:00:00,Plenum,נשים,0.507471,"('0.013*""לקידום"" + 0.013*""מעמד"" + 0.012*""האישה...",4.0
6,532.0,"היו""ר יולי יואל אדלשטיין:",אנחנו נפתח עם נאומים בני דקה. אני מפעיל את ההצ...,נפתח נאומים דקה מפעיל ההצבעה לצורך ההרשמה מוסי...,24,131,2018-02-06 16:00:00,Plenum,חינוך,0.560658,"('0.112*""החינוך"" + 0.033*""התרבות"" + 0.028*""והס...",3.0
7,532.0,"<< יור >> היו""ר יולי יואל אדלשטיין: << יור >>",אני רק מזכיר לחברי הכנסת את חוות הדעת של היועץ...,מזכיר לחברי חוות הדעת היועץ המשפטי לכנסת יושב-...,73,388,2019-10-03 18:00:00,Plenum,נשים,0.597907,"('0.025*""המשפטי"" + 0.018*""מעמד"" + 0.018*""לקידו...",3.0
8,12951.0,"<< דובר >> היו""ר יריב לוין: << דובר >>","לקריאה ראשונה, מטעם הממשלה, הצעת חוק סמכויות מ...",מטעם סמכויות מיוחדות להתמודדות נגיף הקורונה הו...,19,123,2020-09-24 15:00:00,Plenum,קורונה,0.804109,"('0.018*""הפנים"" + 0.018*""הקורונה"" + 0.017*""הור...",1.0
9,12951.0,"<< יור >> היו""ר יריב לוין: << יור >>",<< נושא >> הצעת חוק התוכנית לסיוע כלכלי (נגיף...,<< >> התוכנית לסיוע כלכלי נגיף הקורונה הוראת ה...,22,107,2020-09-29 13:00:00,Plenum,קורונה,0.753701,"('0.023*""הקורונה"" + 0.018*""הוראת"" + 0.014*""העב...",4.0


In [36]:
topics = final_tag_df.Topic.unique()
print("number of topics:", len(topics))
print("topics:", topics)
print("\nnumber of quoted per topic:")
for i,topic in enumerate(topics):
    print(topic, len(final_tag_df[final_tag_df['Topic'] == topic]))
    topic_df = final_tag_df[final_tag_df['Topic'] == topic]
    topic_df = topic_df[['QuoteText', 'Topic', 'Topic_Prob']]
    topic_df.to_csv(f"../data/4 - Auto tagged data/topic_{i}_samples.csv", index=False)

number of topics: 12
topics: ['כלכלי' 'נשים' 'תחבורה' 'קורונה' 'בריאות' 'חינוך' 'רווחה' 'ניצולי שואה'
 'דיור' 'עולים' 'בטחון' 'התנחלויות']

number of quoted per topic:
כלכלי 1220
נשים 1795
תחבורה 755
קורונה 2240
בריאות 1173
חינוך 1837
רווחה 411
ניצולי שואה 425
דיור 299
עולים 1023
בטחון 659
התנחלויות 88
